In [1]:
import os
import time
import random
from pprint import pprint
import numpy as np
import cupy as cp
from pprint import pprint

from load_dataset import custom_lakedepth as ld

from cuml.dask.common import utils as dask_utils
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import dask_cudf
from cuml.dask.ensemble import RandomForestRegressor as cumlDaskRF

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# Import data-visualization modules

In [2]:
# This will use all GPUs on the local host by default
cluster = LocalCUDACluster(threads_per_worker=1)
c = Client(cluster)

# Query the client for all connected workers
workers = c.has_what().keys()
n_workers = len(workers)
n_streams = 8 # Performance optimization

c

data = ld.LakeDepth(42)
cv_train, cv_test, labels_train, labels_test = data.split(0.20)


/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/load_dataset/LakeDepth/pts_merged_final.csv
 - from DATA: reading csv into GPU memory
 - from DATA: done reading csv into GPU memory
 - from DATA: dropped column: FID
 - from DATA: dropped column: Date


In [3]:
n_partitions = n_workers

In [4]:
results = []
TEST_SIZE = 0.20
RANDOM_STATE = 42
# Introduce lists to the hyperparameters we want to

N_ESTIMATORS = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
SPLIT_ALGO = 1
SPLIT_CRITERION = 2
BOOTSTRAP = [True, False]
BOOTSTRAP_FEATURES = False
ROWS_SAMPLE = 1.0
MAX_DEPTH = [int(x) for x in np.linspace(10, 110, num=11)]
MAX_LEAVES = -1
MAX_FEATURES = ['auto', 'sqrt']
N_BINS = [int(x) for x in np.linspace(start=5, stop=20, num=10)]
MIN_ROWS_PER_NODE = 2
MIN_IMPURITY_DECREASE = 0.0
ACCURACY_METRIC = 'mean_ae'  # 'mse' #'r2' # 'median_aw' #
QUANTILEPT = False
SEED = 42
VERBOSE = False

random_grid = {'n_estimators': N_ESTIMATORS,
               'max_depth': MAX_DEPTH,
               'bootstrap': BOOTSTRAP,
               'max_features': MAX_FEATURES,
               'n_bins': N_BINS}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'max_features': ['auto', 'sqrt'],
 'n_bins': [5, 6, 8, 10, 11, 13, 15, 16, 18, 20],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [5]:
rf = cumlDaskRF()

In [6]:
X_dask_cudf = dask_cudf.from_cudf(cv_train, npartitions=n_workers)
y_dask_cudf = dask_cudf.from_cudf(labels_train, npartitions=n_workers)
X_dask_cudf, y_dask_cudf = dask_utils.persist_across_workers(c,
                                                  [X_dask_cudf,
                                                   y_dask_cudf])

In [7]:
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 5, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42)

In [8]:
rf_random.fit(cv_train, labels_train)

ValueError: Attribute _estimator_type does not exist on model <class 'NoneType'>